In [1]:
import numpy as np
from core import DynamicalSystem
from plotting import evolution_plot, phase_portrait, evolution_dot_plot
from rich.console import Console
from rich.traceback import install
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import HoverTool, ColumnDataSource

console = Console()
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x104786cf0>>

In [2]:
def system(t, state_vector, params):
    phi = state_vector
    omega, gamma, Omega, eps, A1, A2, A3 = params
    f1 = A1*np.sin(1*Omega*t-phi-gamma)
    f2 = A2*np.sin(3*Omega*t-phi-gamma)
    f3 = A3*np.sin(5*Omega*t-phi-gamma)
    phi_dot = omega + (eps/np.cos(gamma))*(f1+f2+f3)
    return np.array([phi_dot])

def system_jac(t, state_vector, params):
    phi = state_vector
    _, gamma, Omega, eps, A1, A2, A3 = params
    f1 = -A1*np.cos(1*Omega*t-phi-gamma)
    f2 = -A2*np.cos(3*Omega*t-phi-gamma)
    f3 = -A3*np.cos(5*Omega*t-phi-gamma)

    dfdphi = (eps/np.cos(gamma))*(f1+f2+f3)
    return np.array([dfdphi])

In [10]:
parameters = {
    'omega':    1.0,
    'gamma':    0.1,
    'Omega':    0.9,
    'eps':      0.3,
    'A1':       1.00,
    'A2':       0.00,
    'A3':       0.00
}

# Set initial state and time span
x0 = {'phi': 0.0}

In [11]:
phs = DynamicalSystem(system, t0=0, x0=x0, 
                      parameters=parameters, jac=None)

In [12]:
phs.integrate(10_000,10)

/Users/yernurbaibolatov/.virtualenvs/nld/lib/python3.13/site-packages/scipy/integrate/_ivp/ivp.py:621: UserWarning: The following arguments have no effect for a chosen solver: `jac`.
  solver = method(fun, t0, y0, tf, vectorized=vectorized, **options)


In [13]:
p = figure(title="Time evolution", 
               x_axis_label="time",
               y_axis_label="variables",
               width=1200,
               height=600)
output_notebook()
p.line(phs.t_sol[::1000], phs.x_sol[0][::1000],
            legend_label="phi",
            line_width=2, color='#344966')
    
p.add_tools(HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")]))
p.legend.location = 'top_left'
show(p)

Loading BokehJS ...